In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from utils import missing_values_table, fix_outlier, convert_bytes_to_megabytes, convert_ms_to_seconds




### calling load_data_from_postgres function 

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

from load_data import load_data_from_postgres

In [3]:
query = "SELECT * FROM xdr_data"

In [4]:
df_postgres = load_data_from_postgres(query)
print("Data loaded using psycopg2:")

Data loaded using psycopg2:


In [5]:
df_postgres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

### Extracting columns that are needed for user overview analysis

In [6]:
# List of relevant columns for user overview analysis
user_overview_columns = [
    'IMSI', 'MSISDN/Number', 'IMEI', 'Handset Manufacturer', 'Handset Type',
    'Dur. (ms)', 'Start', 'End', 'Activity Duration DL (ms)', 'Activity Duration UL (ms)',
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]

In [7]:
# Create the df_user_overview DataFrame with the selected columns
df_user_overview = df_postgres[user_overview_columns].copy()

### EDA on the extracted data


In [8]:
df_user_overview.head()

,IMSI,MSISDN/Number,IMEI,Handset Manufacturer,Handset Type,Dur. (ms),Start,End,Activity Duration DL (ms),Activity Duration UL (ms),...,Email DL (Bytes),Email UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,2.082014e+14,3.366496e+10,3.552121e+13,Samsung,Samsung Galaxy A5 Sm-A520F,1823652.0,4/4/2019 12:01,4/25/2019 14:35,37624.0,38787.0,...,3563542.0,137762.0,278082303.0,14344150.0,171744450.0,8814393.0,42.0,5.0,23.0,44.0
1,2.082019e+14,3.368185e+10,3.579401e+13,Samsung,Samsung Galaxy J5 (Sm-J530),1365104.0,4/9/2019 13:04,4/25/2019 8:15,168.0,3560.0,...,629046.0,308339.0,608750074.0,1170709.0,526904238.0,15055145.0,65.0,5.0,16.0,26.0
2,2.082003e+14,3.376063e+10,3.528151e+13,Samsung,Samsung Galaxy A8 (2018),1361762.0,4/9/2019 17:42,4/25/2019 11:58,0.0,0.0,...,2690151.0,672973.0,229584621.0,395630.0,410692588.0,4215763.0,NaN,NaN,6.0,9.0
3,2.082014e+14,3.375034e+10,3.535661e+13,undefined,undefined,1321509.0,4/10/2019 0:31,4/25/2019 7:36,3330.0,37882.0,...,1439754.0,631229.0,799538153.0,10849722.0,749039933.0,12797283.0,NaN,NaN,44.0,44.0
4,2.082014e+14,3.369980e+10,3.540701e+13,Samsung,Samsung Sm-G390F,1089009.0,4/12/2019 20:10,4/25/2019 10:40,0.0,0.0,...,1936496.0,173853.0,527707248.0,3529801.0,550709500.0,13910322.0,NaN,NaN,6.0,9.0


In [9]:
df_user_overview.shape

(150001, 30)

### checking for missing values using imported function missing_values_table

In [10]:
missing_values_table(df_user_overview)


Your selected dataframe has 30 columns.
There are 16 columns that have missing values.


,Missing Values,% of Total Values
Avg RTT DL (ms),27829,18.6
Avg RTT UL (ms),27812,18.5
MSISDN/Number,1066,0.7
IMEI,572,0.4
Handset Manufacturer,572,0.4
Handset Type,572,0.4
IMSI,570,0.4
Dur. (ms),1,0.0
Start,1,0.0
End,1,0.0


## cleaning the data by using different techniques 

In [11]:
df_user_overview.dropna(subset=['MSISDN/Number'], inplace=True)
missing_values_table(df_user_overview)


Your selected dataframe has 30 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Avg RTT DL (ms),27644,18.6
Avg RTT UL (ms),27625,18.5


### Filling the missing Values of the above fields with the mean of the respective columns

In [12]:
# Calculate mean values
mean_rtt_dl = df_user_overview['Avg RTT DL (ms)'].mean()
mean_rtt_ul = df_user_overview['Avg RTT UL (ms)'].mean()

# Fill missing values with mean
df_user_overview['Avg RTT DL (ms)'].fillna(mean_rtt_dl, inplace=True)
df_user_overview['Avg RTT UL (ms)'].fillna(mean_rtt_ul, inplace=True)

## fixing outliers 

In [15]:
# List of relevant columns to apply quartiles(0.95) to fix outliers
user_overview_columns_to_apply_quartiles = [
    'Dur. (ms)',  'Activity Duration DL (ms)', 'Activity Duration UL (ms)',
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'
]

In [16]:
# Apply the fix_outlier function to each specified column
for column in user_overview_columns_to_apply_quartiles:
    if column in df_user_overview.columns:
        df_user_overview = fix_outlier(df_user_overview, column)

In [17]:
df_user_overview.isnull().sum()

IMSI                         0
MSISDN/Number                0
IMEI                         0
Handset Manufacturer         0
Handset Type                 0
Dur. (ms)                    0
Start                        0
End                          0
Activity Duration DL (ms)    0
Activity Duration UL (ms)    0
Total DL (Bytes)             0
Total UL (Bytes)             0
Social Media DL (Bytes)      0
Social Media UL (Bytes)      0
Youtube DL (Bytes)           0
Youtube UL (Bytes)           0
Netflix DL (Bytes)           0
Netflix UL (Bytes)           0
Google DL (Bytes)            0
Google UL (Bytes)            0
Email DL (Bytes)             0
Email UL (Bytes)             0
Gaming DL (Bytes)            0
Gaming UL (Bytes)            0
Other DL (Bytes)             0
Other UL (Bytes)             0
Avg RTT DL (ms)              0
Avg RTT UL (ms)              0
Avg Bearer TP DL (kbps)      0
Avg Bearer TP UL (kbps)      0
dtype: int64

## Formatting the data

### Byte to Megabyte conversion

In [19]:
byte_columns = [
    'Total DL (Bytes)', 'Total UL (Bytes)',
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'YouTube DL (Bytes)', 'YouTube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)'
]

In [20]:
for column in byte_columns:
    if column in df_user_overview.columns:
        df_user_overview[column] = df_user_overview[column].apply(convert_bytes_to_megabytes)

### converting milliseconds to seconds

In [23]:
millisecond_columns = [
    'Dur. (ms)',
    'Activity Duration DL (ms)',
    'Activity Duration UL (ms)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)'
]

In [24]:
for column in millisecond_columns:
    if column in df_user_overview.columns:
        df_user_overview[column] = df_user_overview[column].apply(convert_ms_to_seconds)

In [27]:
df_user_overview.head(5)

,IMSI,MSISDN/Number,IMEI,Handset Manufacturer,Handset Type,Dur. (ms),Start,End,Activity Duration DL (ms),Activity Duration UL (ms),...,Email DL (Bytes),Email UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps)
0,2.082014e+14,3.366496e+10,3.552121e+13,Samsung,Samsung Galaxy A5 Sm-A520F,241.7635,4/4/2019 12:01,4/25/2019 14:35,37.6240,38.7870,...,3.248914,0.131380,265.199950,13.679647,163.788271,8.406060,0.042000,0.005000,23.0,44.0
1,2.082019e+14,3.368185e+10,3.579401e+13,Samsung,Samsung Galaxy J5 (Sm-J530),241.7635,4/9/2019 13:04,4/25/2019 8:15,0.1680,3.5600,...,0.599905,0.294055,580.549311,1.116475,502.495039,14.357705,0.065000,0.005000,16.0,26.0
2,2.082003e+14,3.376063e+10,3.528151e+13,Samsung,Samsung Galaxy A8 (2018),241.7635,4/9/2019 17:42,4/25/2019 11:58,0.1027,0.5897,...,2.565528,0.641797,218.948956,0.794487,391.666973,4.020465,0.108225,0.017637,11.0,18.0
3,2.082014e+14,3.375034e+10,3.535661e+13,undefined,undefined,241.7635,4/10/2019 0:31,4/25/2019 7:36,3.3300,37.8820,...,1.373056,0.601987,762.499002,10.347101,714.340146,12.204440,0.108225,0.017637,44.0,44.0
4,2.082014e+14,3.369980e+10,3.540701e+13,Samsung,Samsung Sm-G390F,241.7635,4/12/2019 20:10,4/25/2019 10:40,0.1027,0.5897,...,1.846786,0.165799,503.260849,3.366281,525.197506,13.265917,0.108225,0.017637,11.0,18.0
